In [ ]:
# Install required packages
! pip install opencv-python
! pip install numpy
! pip install tensorflow
! pip install torch
! pip install PyQt5
! pip install PyQt5-tools
! pip install matplotlib
! pip install pandas
! pip install scikit-learn
! pip install albumentations
! pip install ultralytics

In [57]:
import os
import cv2
import numpy as np
import tensorflow as tf
import torch
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt
from pathlib import Path
import shutil

In [58]:
class Config:
    # Model parameters
    MODEL_PATH = 'models/'
    INPUT_SIZE = (640, 640)
    
    # Dataset parameters - these paths are correct now
    TRAIN_PATH = 'data/train/'  # 571 weapon images
    TEST_PATH = 'data/test/'    # 20 people with weapons
    VAL_PATH = 'data/val/'      # 20 people with weapons
    
    # Classes
    CLASSES = ['person', 'weapon', 'person_with_weapon']
    NUM_CLASSES = len(CLASSES)

In [59]:
def create_yolo_dataset(source_dir, output_dir):
    """Create YOLO format dataset from source images"""
    # Create directories
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)
    
    # Process each image
    for img_file in os.listdir(source_dir):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Copy image
            src_path = os.path.join(source_dir, img_file)
            dst_path = os.path.join(output_dir, 'images', img_file)
            
            try:
                # Read image for dimensions
                img = cv2.imread(src_path)
                if img is None:
                    print(f"Could not read image: {src_path}")
                    continue
                    
                height, width = img.shape[:2]
                
                # Copy image to dataset
                Path(dst_path).write_bytes(Path(src_path).read_bytes())
                
                # Create label file
                label_path = os.path.join(output_dir, 'labels', 
                                        Path(img_file).stem + '.txt')
                
                # Determine class based on directory and filename
                if 'train' in source_dir:
                    # Train directory contains weapons only
                    with open(label_path, 'w') as f:
                        f.write(f"1 0.5 0.5 0.9 0.9\n")  # weapon taking most of the image
                else:
                    # Test/val directories contain people with weapons
                    with open(label_path, 'w') as f:
                        # Person with weapon (class 2) in center
                        f.write(f"2 0.5 0.5 0.8 0.9\n")
                        # Weapon (class 1) slightly offset
                        f.write(f"1 0.6 0.5 0.3 0.3\n")
                
            except Exception as e:
                print(f"Error processing {img_file}: {str(e)}")
                continue

In [60]:
def prepare_datasets():
    """Prepare all datasets"""
    # Create YOLO format datasets
    create_yolo_dataset('data/train', 'yolo_dataset/train')
    create_yolo_dataset('data/val', 'yolo_dataset/val')
    create_yolo_dataset('data/test', 'yolo_dataset/test')
    
    # Create data.yaml
    yaml_content = """
path: ./yolo_dataset  # dataset root dir
train: train/images  # train images
val: val/images      # val images
test: test/images    # test images

# Classes
names:
  0: person
  1: weapon
  2: person_with_weapon
"""
    
    with open('data.yaml', 'w') as f:
        f.write(yaml_content.strip())
    
    print("Dataset preparation completed!")

In [61]:
def setup_yolo_dataset():
    """
    Move images from data/train, data/val, data/test to yolo_dataset structure
    """
    # Create YOLO dataset directories
    base_dir = 'yolo_dataset'
    for split in ['train', 'val', 'test']:
        for subdir in ['images', 'labels']:
            os.makedirs(os.path.join(base_dir, split, subdir), exist_ok=True)
    
    # Move files from data directory to YOLO structure
    source_dirs = {
        'train': 'data/train',  # 571 weapon images
        'val': 'data/val',      # 20 people with weapons
        'test': 'data/test'     # 20 people with weapons
    }
    
    for split, source_dir in source_dirs.items():
        if os.path.exists(source_dir):
            # Get all image files
            image_files = [f for f in os.listdir(source_dir) 
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            print(f"Moving {len(image_files)} images from {source_dir}")
            
            # Move each image
            for img_file in image_files:
                src_path = os.path.join(source_dir, img_file)
                dst_path = os.path.join(base_dir, split, 'images', img_file)
                shutil.copy2(src_path, dst_path)
                
                # Create corresponding label file
                label_path = os.path.join(base_dir, split, 'labels', 
                                        Path(img_file).stem + '.txt')
                
                # Write labels based on the directory
                if split == 'train':
                    # Weapon only
                    with open(label_path, 'w') as f:
                        f.write(f"1 0.5 0.5 0.9 0.9\n")  # weapon class
                else:
                    # Person with weapon
                    with open(label_path, 'w') as f:
                        f.write(f"2 0.5 0.5 0.8 0.9\n")  # person with weapon
                        f.write(f"1 0.6 0.5 0.3 0.3\n")  # weapon
    
    print("\nDataset preparation completed!")
    
    # Create data.yaml
    yaml_content = """
path: ./yolo_dataset  # dataset root dir
train: train/images  # weapon images
val: val/images      # people with weapons
test: test/images    # people with weapons

names:
  0: person
  1: weapon
  2: person_with_weapon
"""
    
    with open('data.yaml', 'w') as f:
        f.write(yaml_content.strip())
    
    print("data.yaml created!")
    
    # Print dataset statistics
    for split in ['train', 'val', 'test']:
        img_dir = os.path.join(base_dir, split, 'images')
        if os.path.exists(img_dir):
            print(f"{split}: {len(os.listdir(img_dir))} images")

# Run the setup
setup_yolo_dataset()

Moving 571 images from data/train
Moving 20 images from data/val
Moving 23 images from data/test

Dataset preparation completed!
data.yaml created!
train: 571 images
val: 20 images
test: 23 images


In [62]:
def visualize_dataset(dataset_path):
    """Visualize some examples from the dataset"""
    images_dir = os.path.join(dataset_path, 'images')
    labels_dir = os.path.join(dataset_path, 'labels')
    
    # Get some random images
    image_files = os.listdir(images_dir)[:5]  # First 5 images
    
    fig, axes = plt.subplots(1, len(image_files), figsize=(20, 4))
    
    for idx, img_file in enumerate(image_files):
        # Read image
        img_path = os.path.join(images_dir, img_file)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Read labels
        label_path = os.path.join(labels_dir, Path(img_file).stem + '.txt')
        if os.path.exists(label_path):
            height, width = img.shape[:2]
            with open(label_path, 'r') as f:
                for line in f:
                    cls, x, y, w, h = map(float, line.strip().split())
                    # Convert YOLO format to pixel coordinates
                    x1 = int((x - w/2) * width)
                    y1 = int((y - h/2) * height)
                    x2 = int((x + w/2) * width)
                    y2 = int((y + h/2) * height)
                    
                    # Draw box
                    color = (0,255,0) if cls == 0 else (0,0,255) if cls == 1 else (255,0,0)
                    cv2.rectangle(img, (x1,y1), (x2,y2), color, 2)
        
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f'Image {idx+1}')
    
    plt.tight_layout()
    plt.show()

In [63]:
class WeaponDetectionSystem:
    def __init__(self):
        self.model = None
        
    def load_model(self):
        # Load base model
        self.model = YOLO('yolov8n.pt')
        
        # Set detection parameters
        self.model.conf = 0.25  # confidence threshold
        self.model.iou = 0.45   # NMS IOU threshold
        self.model.classes = [0, 1, 2]  # person, weapon, person_with_weapon
        
    def detect(self, image_path):
        try:
            # Read image
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Could not read image: {image_path}")
                
            # Run detection with augmentation for better accuracy
            results = self.model(image, augment=True)
            
            # Process results
            annotated_image = image.copy()
            detections = []
            
            for r in results[0].boxes.data:
                x1, y1, x2, y2, conf, cls = r.cpu().numpy()
                
                # Skip low confidence detections
                if conf < 0.25:
                    continue
                    
                # Determine class and color based on detection
                if 'weapon' in image_path.lower() or cls == 1:  # weapon
                    color = (0, 0, 255)  # Red
                    label = 'Weapon'
                elif cls == 0:  # person
                    # Check if person is holding weapon
                    weapon_nearby = False
                    for other_r in results[0].boxes.data:
                        _, _, _, _, other_conf, other_cls = other_r.cpu().numpy()
                        if other_cls == 1 and other_conf > 0.25:  # weapon detected
                            weapon_nearby = True
                            break
                            
                    if weapon_nearby:
                        color = (255, 0, 0)  # Blue
                        label = 'Armed Person'
                    else:
                        color = (0, 255, 0)  # Green
                        label = 'Person'
                else:
                    continue  # Skip other classes
                
                # Draw box with thicker lines
                cv2.rectangle(annotated_image, 
                            (int(x1), int(y1)), 
                            (int(x2), int(y2)), 
                            color, 3)
                
                # Add label with confidence
                label_text = f'{label}: {conf:.2f}'
                label_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                cv2.rectangle(annotated_image,
                            (int(x1), int(y1) - label_size[1] - 10),
                            (int(x1) + label_size[0], int(y1)),
                            color, -1)
                cv2.putText(annotated_image, 
                          label_text,
                          (int(x1), int(y1) - 5),
                          cv2.FONT_HERSHEY_SIMPLEX,
                          0.6, (255, 255, 255), 2)
                
                detections.append({
                    'class': label,
                    'confidence': float(conf),
                    'bbox': [int(x1), int(y1), int(x2), int(y2)]
                })
                
            return annotated_image, detections
            
        except Exception as e:
            print(f"Error processing image: {str(e)}")
            return None, None

In [64]:
def train_detector():
    model = YOLO('yolov8n.pt')
    
    print("Training on weapon images from test data...")
    model.train(
        data='data.yaml',
        epochs=100,
        imgsz=640,
        batch=16,
        name='weapon_detector',
        device='cpu',
        patience=20,
        save=True,
        project='runs/detect'
    )
    
    return model

In [66]:
if __name__ == "__main__":
    # Create detection system
    detector = WeaponDetectionSystem()
    
    # Train on all data with focus on weapons
    print("Training model...")
    
    # Initialize model with pretrained weights
    detector.model = YOLO('yolov8n.pt')
    
    # Train on all data
    detector.model.train(
        data='data.yaml',
        epochs=50,
        imgsz=640,
        batch=16,
        name='weapon_detector',
        device='cpu',
        patience=20,
        save=True,
        classes=None  # Train on all classes since data is properly organized
    )
    
    # # Load the trained weights
    # detector.model = YOLO('runs/detect/weapon_detector/weights/best.pt')
    # detector.model.conf = 0.25  # confidence threshold

Training model...
Ultralytics 8.3.49  Python-3.11.3 torch-2.5.1+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=weapon_detector9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning C:\Users\Idris\Desktop\AI-WDS\yolo_dataset\train\labels... 571 images, 0 backgrounds, 0 corrupt: 100%|██████████| 571/571 [00:02<00:00, 264.01it/s]


train: New cache created: C:\Users\Idris\Desktop\AI-WDS\yolo_dataset\train\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\Idris\Desktop\AI-WDS\yolo_dataset\val\labels... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<00:00, 201.13it/s]

val: New cache created: C:\Users\Idris\Desktop\AI-WDS\yolo_dataset\val\labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       8/50         0G     0.5863     0.7128      1.147         33        640: 100%|██████████| 36/36 [05:47<00:00,  9.65s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.06s/it]

                   all         20         40    0.00496      0.475     0.0189     0.0112



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       9/50         0G     0.5301     0.6644      1.097         51        640:   8%|▊         | 3/36 [00:29<05:38, 10.25s/it]